In [3]:
import requests

data = requests.get('http://localhost:8080/users').json()
print(data)

[[6, 'Danny Torrances Finger'], [2, 'Leia Organa'], [3, 'Tony Stark'], [4, 'Ellen Ripley'], [5, 'J. Jonah Jameson'], [7, 'Danny Torrance'], [0, 'John Wick'], [1, 'Mike Wazowski'], [8, 'Mario Bru'], [9, 'Felipe Ferrero'], [10, 'Marc Pomar'], [11, 'Julio Porto']]


### Create User:

In [25]:
new_name = {'name': 'Paquito'}
userid = requests.post("http://localhost:8080/user/create", data=new_name).json()
print(userid)

14


### Create chat:

In [9]:
chatid = requests.post("http://localhost:8080/chat/create").json()
print(chatid)

{'chatid': 10}


### Add Message:

In [44]:
new_message = {'userid': 11, 'message': 'Lets go to have lunch'}
requests.post("http://localhost:8080/chat/0/addmessage", data=new_message)

<Response [200]>

### Get messages in chat:

In [31]:
messages = requests.get('http://localhost:8080/chat/0/list').json()
print(messages)

[['Hey Mike, whats up??'], ['Dude!!! 😀️ Did you watch the game last night?'], ['No, had to work. How was it?'], ['Awesome! Boyander scored 3 goals!'], ['No way! 😮️'], ['Way...'], ['I bet people went crazy'], ['For sure. We stayed at KuboLoco until 6AM.'], ['Motherfucker! Hungover? 😵️'], ['Dead, burried and still suffering! \U0001f92e️'], ['Paracetamol is your best friend :P'], ['Boyander is the fucking master!']]


### Get messages from user:

In [23]:
messages = requests.get('http://localhost:8080/user/13/listmessages').json()
print(messages)

[['Boyander is the fucking master!']]


### Sentiment analysis:

In [41]:
import nltk # install nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

sid = SentimentIntensityAnalyzer()

sid.polarity_scores("Happy")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mariobru/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5719}

In [43]:
for e in messages:
    print(*e)
    print(sid.polarity_scores(*e))
    print(TextBlob(*e).sentiment)

Hey Mike, whats up??
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentiment(polarity=0.0, subjectivity=0.0)
Dude!!! 😀️ Did you watch the game last night?
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentiment(polarity=-0.2, subjectivity=0.23333333333333334)
No, had to work. How was it?
{'neg': 0.268, 'neu': 0.732, 'pos': 0.0, 'compound': -0.296}
Sentiment(polarity=0.0, subjectivity=0.0)
Awesome! Boyander scored 3 goals!
{'neg': 0.0, 'neu': 0.39, 'pos': 0.61, 'compound': 0.6892}
Sentiment(polarity=1.0, subjectivity=1.0)
No way! 😮️
{'neg': 0.555, 'neu': 0.445, 'pos': 0.0, 'compound': -0.3595}
Sentiment(polarity=0.0, subjectivity=0.0)
Way...
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Sentiment(polarity=0.0, subjectivity=0.0)
I bet people went crazy
{'neg': 0.444, 'neu': 0.556, 'pos': 0.0, 'compound': -0.34}
Sentiment(polarity=-0.6, subjectivity=0.9)
For sure. We stayed at KuboLoco until 6AM.
{'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'compound': 0.3182}
Sentimen